In [1]:
import numpy as np
import pandas as pd
import math
from scipy.stats import t
import matplotlib.pyplot as plt
from collections import deque

In [2]:
class LehmerRNG:
  def __init__(self, seed = 123456789, a=48271, m=2**31-1):
    self.seed = seed
    self.a = a
    self.m = m

  def next(self):
    self.seed=(self.a*self.seed) % self.m
    return self.seed/self.m   # per normalizzare in (0,1)

# Campionamento Trasformata Inversa per distribuzione esponenziale
def exponential_sample(rng, mean):
  u = rng.next()
  return -mean * math.log(1-u)

In [3]:
import math
import pandas as pd
from collections import deque

def simulate_ps_ABAPA(
    lmb,                    # tasso arrivi (costante su tutto il sistema)
    mu_A1, mu_B, mu_A2, mu_P, mu_A3,   # tassi di servizio per visita
    threshold,              # genera arrivi fino a 'threshold', poi svuota
    rng,                    # RNG con metodo .next() -> U(0,1)
    max_parallel=5,         # cap per stazione (PS solo tra gli attivi)
    eps=1e-12
):
    #  helper exp(mean) che usa rng.next()
    def exp_from_mean(mean):
        if mean <= 0:
            return math.inf
        u = rng.next()
        if u <= 0.0:
            u = 1e-15
        return -mean * math.log(1.0 - u)

    # tre server reali: A, B, P
    servers = {
        "A": {"name": "A", "c": 1, "max_parallel": int(max_parallel),
              "active": [], "queue": deque(), "N_time_accum": 0.0},
        "B": {"name": "B", "c": 1, "max_parallel": int(max_parallel),
              "active": [], "queue": deque(), "N_time_accum": 0.0},
        "P": {"name": "P", "c": 1, "max_parallel": int(max_parallel),
              "active": [], "queue": deque(), "N_time_accum": 0.0},
    }

    #  sequenza logica delle visite (5 tappe), ma i server usati sono 3
    def stage_server_name(pos):
        return ["A","B","A","P","A"][pos-1]

    def a_visit_index_after(pos):
        return [1,1,2,2,3][pos-1]

    def draw_demand_for(pos):
        if pos == 1:
            return exp_from_mean(1.0/float(mu_A1))
        elif pos == 2:
            return exp_from_mean(1.0/float(mu_B))
        elif pos == 3:
            return exp_from_mean(1.0/float(mu_A2))
        elif pos == 4:
            return exp_from_mean(1.0/float(mu_P))
        else:
            return exp_from_mean(1.0/float(mu_A3))

    #  stato globale
    t = 0.0
    mean_interarrival = 1.0 / float(lmb)
    next_arrival = t + exp_from_mean(mean_interarrival)

    job_id = 0
    jobs_rows = []
    jobs_info = {}

    total_count = 0
    lost_count  = 0
    loss_on_b = 0
    loss_on_p = 0

    def admit_or_enqueue(server, seg, now):
        if len(server["active"]) < server["max_parallel"]:
            seg["admitted"] = now
            seg["N_at_Admit"] = len(server["active"]) + 1
            server["active"].append(seg)
        else:
            server["queue"].append(seg)

    def any_work_left():
        arrivals_ok = (next_arrival is not None and next_arrival <= threshold)
        act = any(len(s["active"]) > 0 for s in servers.values())
        wq  = any(len(s["queue"])  > 0 for s in servers.values())
        return arrivals_ok or act or wq

    def per_job_rate(c, n):
        return (c / n) if n > 0 else 0.0

    while any_work_left():
        # 1) prossimo completion
        next_compl_time = math.inf
        compl_server_key = None
        for key, srv in servers.items():
            n = len(srv["active"])
            if n <= 0: continue
            r_min = min(seg["remaining"] for seg in srv["active"])
            dt = (r_min * n) / srv["c"]
            tc = t + dt
            if tc < next_compl_time:
                next_compl_time = tc
                compl_server_key = key

        # 2) prossimo arrivo valido
        ta = next_arrival if (next_arrival is not None and next_arrival <= threshold) else math.inf

        # 3) selezione evento
        t_next = min(ta, next_compl_time)
        if math.isinf(t_next): break

        # 4) avanzamento tempo e consumo lavoro
        delta = t_next - t
        if delta > 0:
            # accumula numero di job * tempo
            for srv in servers.values():
                srv["N_time_accum"] += len(srv["active"]) * delta

            # riduci i lavori attivi
            for srv in servers.values():
                n = len(srv["active"])
                if n <= 0: continue
                rate = per_job_rate(srv["c"], n)
                dj = rate * delta
                for seg in srv["active"]:
                    seg["time_acc"] += delta
                    seg["twN_acc"]  += delta * n
                    seg["remaining"] -= dj
                    if seg["remaining"] < 0.0:
                        seg["remaining"] = 0.0
        t = t_next

        # 5) evento
        if t == ta:
            # ARRIVO
            job_id += 1
            total_count += 1
            jobs_info[job_id] = {"arrival0": t, "stage_recs": [], "demand_total": 0.0}
            pos = 1
            server_name = stage_server_name(pos)
            srv = servers[server_name]
            demand = draw_demand_for(pos)
            seg = {"job_id": job_id, "pos": pos, "server": server_name,
                   "a_visit": a_visit_index_after(pos), "arrival_stage": t,
                   "admitted": None, "remaining": demand, "demand": demand,
                   "time_acc": 0.0, "twN_acc": 0.0, "N_at_Admit": None}
            admit_or_enqueue(srv, seg, t)
            ia = exp_from_mean(mean_interarrival)
            next_arrival = (t + ia) if (t + ia) <= threshold else None

        else:
            # COMPLETION
            srv = servers[compl_server_key]
            finished = [seg for seg in srv["active"] if seg["remaining"] <= eps]
            if finished:
                srv["active"] = [seg for seg in srv["active"] if seg["remaining"] > eps]
                for seg in finished:
                    jid = seg["job_id"]
                    job = jobs_info[jid]
                    pos = seg["pos"]
                    server_name = seg["server"]
                    IN = seg["admitted"]
                    OUT = t
                    Demand = seg["demand"]
                    Wait = (seg["admitted"] - seg["arrival_stage"]) if seg["admitted"] else 0.0
                    Service = OUT - IN if IN is not None else 0.0
                    N_at_Admit = seg["N_at_Admit"] if seg["N_at_Admit"] else 0
                    AvgN = (seg["twN_acc"] / seg["time_acc"]) if seg["time_acc"] > 0 else float(N_at_Admit)

                    job["stage_recs"].append({
                        "pos": pos, "server": server_name, "id_label": f"{jid}[{seg['a_visit']}]",
                        "IN": IN, "OUT": OUT, "Demand": Demand, "Wait": Wait,
                        "Service": Service, "N_at_Admit": N_at_Admit, "AvgN_during": AvgN
                    })
                    job["demand_total"] += Demand

                    # prosegui o chiudi il job
                    if pos < 5:
                        pos2 = pos + 1
                        server2 = stage_server_name(pos2)
                        srv2 = servers[server2]
                        demand2 = draw_demand_for(pos2)
                        seg2 = {"job_id": jid, "pos": pos2, "server": server2,
                                "a_visit": a_visit_index_after(pos2), "arrival_stage": t,
                                "admitted": None, "remaining": demand2, "demand": demand2,
                                "time_acc": 0.0, "twN_acc": 0.0, "N_at_Admit": None}
                        admit_or_enqueue(srv2, seg2, t)
                    else:
                        row = {"OrigID": jid, "Arrival0": job["arrival0"],
                               "Completion": t, "SojournTotal": t-job["arrival0"],
                               "DemandTotal": job["demand_total"]}
                        recs = sorted(job["stage_recs"], key=lambda r: r["pos"])
                        for i, rec in enumerate(recs, start=1):
                            row.update({f"Server_{i}": rec["server"], f"ID_{i}": rec["id_label"],
                                        f"IN_{i}": rec["IN"], f"OUT_{i}": rec["OUT"],
                                        f"Demand_{i}": rec["Demand"], f"Wait_{i}": rec["Wait"],
                                        f"Service_{i}": rec["Service"], f"N_at_Admit_{i}": rec["N_at_Admit"],
                                        f"AvgN_during_{i}": rec["AvgN_during"]})
                        jobs_rows.append(row)

                while len(srv["active"]) < srv["max_parallel"] and srv["queue"]:
                    qseg = srv["queue"].popleft()
                    qseg["admitted"]   = t
                    qseg["N_at_Admit"] = len(srv["active"]) + 1
                    srv["active"].append(qseg)

    #  DataFrame finale
    K = 5
    base_cols = ["OrigID","Arrival0","Completion","SojournTotal","DemandTotal"]
    per_stage_cols = []
    for i in range(1, K+1):
        per_stage_cols += [
            f"Server_{i}", f"ID_{i}", f"IN_{i}", f"OUT_{i}", f"Demand_{i}",
            f"Wait_{i}", f"Service_{i}", f"N_at_Admit_{i}", f"AvgN_during_{i}"
        ]

    df = pd.DataFrame(jobs_rows)
    for c in base_cols + per_stage_cols:
        if c not in df.columns:
            df[c] = pd.NA
    df = df[base_cols + per_stage_cols].sort_values("OrigID").reset_index(drop=True)

    # Calcolo globale AvgN
    AvgN_A = servers["A"]["N_time_accum"] / t if t > 0 else 0.0
    AvgN_B = servers["B"]["N_time_accum"] / t if t > 0 else 0.0
    AvgN_P = servers["P"]["N_time_accum"] / t if t > 0 else 0.0

    return df, {"AvgN_A": AvgN_A, "AvgN_B": AvgN_B, "AvgN_P": AvgN_P}


In [7]:
rng = LehmerRNG(seed=42)



df_jobs,b = simulate_ps_ABAPA(
    lmb=0.5,
    mu_A1=5.0, mu_B=1.25, mu_A2=2.5, mu_P=2.5, mu_A3=10,
    threshold=20000,
    rng=rng,
    max_parallel=1000000000
)



print("Jobs (prime 12):")
print(df_jobs.head(200).to_string(index=False))

KeyboardInterrupt: 

In [8]:
import math, heapq
import pandas as pd

def simulate_ps_ABAPA_fast(
    lmb,                    # tasso arrivi (Poisson)
    mu_A1, mu_B, mu_A2, mu_P, mu_A3,   # tassi di servizio per visita (Exp con rate mu)
    threshold,              # orizzonte temporale per GENERARE arrivi (secondi)
    rng,                    # RNG con metodo .next() -> U(0,1)
    warmup_time=0.0,        # scarta completamenti < warmup_time dalle stats
    record_stride=10_000,   # registra 1 job ogni N (riduci I/O e RAM)
    record_max=200,         # al massimo M job nel DataFrame
    eps=1e-12
):
    # -------- helpers Exp(rate) ----------
    def exp_from_rate(rate):
        if rate <= 0: 
            return math.inf
        u = rng.next()
        if u <= 0.0: u = 1e-15
        return -math.log(1.0 - u) / rate

    # -------- mappa stadi ABAPA ----------
    # pos: 1..5  -> server: A, B, A, P, A
    def stage_server_name(pos): return ("A","B","A","P","A")[pos-1]
    def a_visit_index_after(pos): return (1,1,2,2,3)[pos-1]  # label per ID_[i]
    def draw_demand(pos):
        if pos == 1: return exp_from_rate(mu_A1)
        if pos == 2: return exp_from_rate(mu_B)
        if pos == 3: return exp_from_rate(mu_A2)
        if pos == 4: return exp_from_rate(mu_P)
        return exp_from_rate(mu_A3)

    # -------- server PS con Virtual Time + heap --------
    def new_server(name, c=1.0):
        return {
            "name": name,
            "c": float(c),
            "V": 0.0,               # virtual time
            "N": 0,                 # attivi
            "heap": [],             # [(deadline, seg_dict)]
            "N_time_accum": 0.0,    # integrale N(t) dt
        }

    servers = { "A": new_server("A"), "B": new_server("B"), "P": new_server("P") }

    # -------- stato globale --------
    t = 0.0
    next_arrival = exp_from_rate(lmb)  # primo arrivo
    # id lavoro globale
    job_id = 0
    # campionamento & stats
    recorded = 0
    sample_jobs = {}  # jid -> {"arrival0":..., "stage_recs":[], "demand_total":0.0}
    rows = []         # righe finali DataFrame (solo campionati)

    # stats streaming su completamenti job (post warmup)
    comp_count = 0
    sum_soj = 0.0
    sum_soj2 = 0.0

    # ---- utility per avanzamento tempo (aggiorna V e N_time_accum) ----
    def advance_all(delta):
        if delta <= 0: 
            return
        for s in servers.values():
            s["N_time_accum"] += s["N"] * delta
            if s["N"] > 0:
                s["V"] += (s["c"] / s["N"]) * delta

    # ---- ammissione PS pura: nessuna coda ----
    def admit(server_key, seg, now):
        s = servers[server_key]
        seg["admitted"] = now
        seg["N_at_Admit"] = s["N"] + 1
        # deadline in virtual time
        D = s["V"] + seg["demand"]
        heapq.heappush(s["heap"], (D, seg))
        s["N"] += 1

    # ---- tempo reale al prossimo completion di un server ----
    def server_next_completion_time(s, now):
        if s["N"] <= 0 or not s["heap"]:
            return math.inf
        D_min = s["heap"][0][0]
        dV = D_min - s["V"]
        if dV <= eps:
            return now  # completamento immediato
        # dV = (c/N) * dt  ->  dt = dV * N / c
        dt = dV * s["N"] / s["c"]
        return now + dt

    # ---- generazione arrivi finché t <= threshold; poi svuota ----
    def arrivals_enabled(now):
        return (next_arrival is not None) and (next_arrival <= threshold)

    # ---- loop eventi (arrivi o completamenti) ----
    while True:
        any_active = any(s["N"] > 0 for s in servers.values())
        if not arrivals_enabled(t) and not any_active:
            break

        # prossimo completion globale
        t_next_comp = math.inf
        comp_srv_key = None
        for key, s in servers.items():
            tc = server_next_completion_time(s, t)
            if tc < t_next_comp:
                t_next_comp = tc
                comp_srv_key = key

        # prossimo arrivo valido
        ta = next_arrival if arrivals_enabled(t) else math.inf

        # prossimo evento
        t_next = ta if ta < t_next_comp else t_next_comp
        if math.isinf(t_next):
            break

        # avanza tempo, aggiorna V e integrali
        delta = t_next - t
        if delta < 0: delta = 0.0
        advance_all(delta)
        t = t_next

        if t == ta:
            # ===== ARRIVO =====
            job_id += 1
            # decidi subito se campionare questo job
            sample_this = (recorded < record_max) and (job_id % max(1, record_stride) == 0)
            if sample_this:
                sample_jobs[job_id] = {"arrival0": t, "stage_recs": [], "demand_total": 0.0}

            # primo stadio su A
            demand = draw_demand(1)
            seg = {
                "job_id": job_id, "pos": 1, "server": "A",
                "a_visit": a_visit_index_after(1),
                "arrival_stage": t, "admitted": None, "demand": demand,
                "N_at_Admit": None, "sample": sample_this
            }
            admit("A", seg, t)
            # schedule prossimo arrivo
            nxt = t + exp_from_rate(lmb)
            next_arrival = nxt if nxt <= threshold else None

        else:
            # ===== COMPLETION su comp_srv_key =====
            s = servers[comp_srv_key]

            # possono completare più segmenti in questo identico istante (stessa deadline)
            while s["N"] > 0 and s["heap"] and (s["heap"][0][0] - s["V"] <= eps):
                _, seg = heapq.heappop(s["heap"])
                s["N"] -= 1
                jid = seg["job_id"]
                pos = seg["pos"]
                server_name = seg["server"]
                IN = seg["admitted"]
                OUT = t
                Demand = seg["demand"]
                Wait = (IN - seg["arrival_stage"]) if IN is not None else 0.0
                Service = OUT - IN if IN is not None else 0.0
                N_at_Admit = seg["N_at_Admit"] or 0

                # aggiorna eventuale job campionato
                if seg.get("sample"):
                    js = sample_jobs.get(jid)
                    if js is not None:
                        js["stage_recs"].append({
                            "pos": pos, "server": server_name, "id_label": f"{jid}[{seg['a_visit']}]",
                            "IN": IN, "OUT": OUT, "Demand": Demand, "Wait": Wait,
                            "Service": Service, "N_at_Admit": N_at_Admit
                        })
                        js["demand_total"] += Demand

                # prosegui o chiudi job
                if pos < 5:
                    pos2 = pos + 1
                    server2 = stage_server_name(pos2)
                    demand2 = draw_demand(pos2)
                    seg2 = {
                        "job_id": jid, "pos": pos2, "server": server2,
                        "a_visit": a_visit_index_after(pos2),
                        "arrival_stage": t, "admitted": None, "demand": demand2,
                        "N_at_Admit": None, "sample": seg.get("sample", False)
                    }
                    admit(server2, seg2, t)
                else:
                    # completamento job
                    # stats streaming (post warmup time)
                    if OUT >= warmup_time:
                        comp_count += 1
                        soj = OUT - (sample_jobs[jid]["arrival0"] if seg.get("sample") else seg["arrival_stage"])
                        # Nota: l'arrival0 del campionato è esatto; per i non campionati non lo conserviamo,
                        # quindi come fallback usiamo l'arrivo del 5° stadio che coincide con l'OUT del 4° stadio.
                        # Se vuoi sojourn esatto per tutti, abilita anche "track_arrival0_for_all=True" (costo memoria).
                        sum_soj += soj
                        sum_soj2 += soj * soj

                    # se era campionato, crea riga
                    if seg.get("sample"):
                        js = sample_jobs.pop(jid, None)
                        if js and recorded < record_max:
                            recs = sorted(js["stage_recs"], key=lambda r: r["pos"])
                            row = {"OrigID": jid, "Arrival0": js["arrival0"],
                                   "Completion": OUT, "SojournTotal": OUT - js["arrival0"],
                                   "DemandTotal": js["demand_total"]}
                            for i, rec in enumerate(recs, start=1):
                                row.update({
                                    f"Server_{i}": rec["server"], f"ID_{i}": rec["id_label"],
                                    f"IN_{i}": rec["IN"], f"OUT_{i}": rec["OUT"],
                                    f"Demand_{i}": rec["Demand"], f"Wait_{i}": rec["Wait"],
                                    f"Service_{i}": rec["Service"], f"N_at_Admit_{i}": rec["N_at_Admit"],
                                })
                            rows.append(row)
                            recorded += 1

    # ---- chiusura: DataFrame campionato + metriche globali ----
    K = 5
    base_cols = ["OrigID","Arrival0","Completion","SojournTotal","DemandTotal"]
    per_stage_cols = []
    for i in range(1, K+1):
        per_stage_cols += [
            f"Server_{i}", f"ID_{i}", f"IN_{i}", f"OUT_{i}", f"Demand_{i}",
            f"Wait_{i}", f"Service_{i}", f"N_at_Admit_{i}"
        ]
    df = pd.DataFrame(rows) if rows else pd.DataFrame(columns=base_cols+per_stage_cols)
    for c in base_cols + per_stage_cols:
        if c not in df.columns:
            df[c] = pd.NA
    if len(df) > 0:
        df = df[base_cols + per_stage_cols].sort_values("OrigID").reset_index(drop=True)

    t_end = t if t > 0 else 1.0
    stats = {
        "t_end": t_end,
        "arrivals_generated": job_id,
        "jobs_completed_after_warmup": comp_count,
        "AvgN_A": servers["A"]["N_time_accum"] / t_end,
        "AvgN_B": servers["B"]["N_time_accum"] / t_end,
        "AvgN_P": servers["P"]["N_time_accum"] / t_end,
    }
    if comp_count > 0:
        mean_soj = sum_soj / comp_count
        var_soj = (sum_soj2 / comp_count) - (mean_soj**2)
        stats.update({
            "mean_sojourn": mean_soj,
            "std_sojourn": math.sqrt(max(0.0, var_soj)),
            "throughput": comp_count / max(1e-12, (t_end - warmup_time)),
        })
    else:
        stats.update({"mean_sojourn": float("nan"), "std_sojourn": float("nan"), "throughput": 0.0})

    return df, stats


In [10]:
# rng = LehmerRNG(seed=42)  # come il tuo
df_steady, stats_steady = simulate_ps_ABAPA_fast(
    lmb=0.5,
    mu_A1=5.0, mu_B=1.25, mu_A2=2.5, mu_P=2.5, mu_A3=10.0,
    threshold=20000,   # tempo di generazione arrivi
    rng=rng,
    warmup_time=1_000,  # scarta il transiente iniziale dalle stats
    record_stride=5_000,
    record_max=200
)
print(stats_steady)
print(df_steady.head(20).to_string(index=False))


KeyboardInterrupt: 